In [70]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/upload", methods=["GET", "POST"])
def upload_file():
    if request.method == "POST" :
        
        file_name = request.form.get("fileName")
        file_extension = request.form.get("fileExtension")
        user_id = request.form.get("user_id")
        orign_file = f"{file_name}.{file_extension}"
        
        print(file_name)
        print(file_extension)
        print(orign_file)
        print(user_id)
        return "성공"
    else :
        print(request.args.get("fileName"))
        return "get방식"
   
if __name__ == '__main__':
    app.run(host="127.0.0.1", port="9000")


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2023 12:39:47] "POST /upload HTTP/1.1" 200 -


7_data
pkl
7_data.pkl
123


In [62]:
import os

def read_data_from_file(full_file_name):
    file_folder = '../Desktop/user_ecg/'
    file_path = os.path.join(file_folder, full_file_name)

    with open(file_path, "r") as file:
        data = file.read()

    return data

In [74]:
def main(orign_file, user_id) : 

    from typing import Dict, Union
    import pickle
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import numpy as np
    from collections import OrderedDict
    import glob
    from resnet1d import SimpleResNet1D
    from unet1d import SimpleUNet1D

    def load_ecg_pickle(ecg_path:str)->np.ndarray:
        with open(ecg_path, "rb") as f:
            ecg = pickle.load(f)
        out = []
        for key in ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']:
            out.append(ecg[key])
        ecg = np.stack(out, axis=0)
        return ecg


    class ECG_Infer_Module():
        def __init__(self, resnet_path, unet_path, num_leads=12, out_classes=["af", "sr", "sb", "gsvt"]):
            self.num_leads = num_leads
            self.out_classes = out_classes
            self.num_classes = len(out_classes)
            self.resnet = SimpleResNet1D(in_channels=num_leads, num_classes=self.num_classes)
            self.unet = SimpleUNet1D(in_channels=num_leads, out_channels=self.num_classes)
            self.load_model(resnet_path, unet_path)

        def modify_checkpoint(self, checkpoint):
            temp_dict = OrderedDict()
            for old_key in checkpoint:
                new_key = old_key[7:]
                temp_dict[new_key] = checkpoint[old_key]
            return temp_dict


        def load_model(self, resnet_path, unet_path, trained_from_gpu=True):  # XXX need to set device
            device = torch.device('cpu')
            if resnet_path:
                checkpoint = torch.load(resnet_path, map_location=device )
                if trained_from_gpu:
                    checkpoint = self.modify_checkpoint(checkpoint)
                self.resnet.load_state_dict(checkpoint)
            if unet_path:
                checkpoint = torch.load(unet_path, map_location=device )
                if trained_from_gpu:
                    checkpoint = self.modify_checkpoint(checkpoint)
                self.unet.load_state_dict(torch.load(unet_path))
            self.resnet.to('cpu')
            self.resnet.eval()
            self.unet.to('cpu')
            self.unet.eval()


        def post_process_resnet(self, resnet_output:np.ndarray)->Dict[str, bool]:
            idx = torch.argmax(resnet_output)
            out_dict = {}
            for i, class_name in enumerate(self.out_classes):
                if idx == i:
                    out_dict[class_name] = True
                else:
                    out_dict[class_name] = False
            return out_dict


        def post_process_unet(self, unet_ouput:np.ndarray)->Dict[str, float]:
            out_dict = {}
            mean_dict = {
                "hr": 89,
                "pr": 140,
                "qrs": 74,
                "qt": 330,
                "qtc": 400
            }
            for class_name in mean_dict:
                out_dict[class_name] = np.random.normal(loc=mean_dict[class_name])
            return out_dict


        def infer(self, ecg:np.ndarray)->Dict[str, Union[bool, str]]:
            ecg = torch.Tensor(ecg).unsqueeze(0)  # to torch.Tensor with shape (1, 12, 5000)
            resnet_output = self.resnet(ecg)
            resnet_output = self.post_process_resnet(resnet_output)
            unet_output = self.post_process_unet(None)
            out_dict = {**resnet_output, **unet_output}

            return out_dict


    if __name__ == "__main__":
        ecg_pkl_list = glob.glob(f"../Desktop/user_ecg/4_data.pkl")  # sample ecg data
        ecg_pkl_list.sort()


        ecg_infer_module = ECG_Infer_Module("", "")  # no module yet

        for ecg in ecg_pkl_list:
            ecg = load_ecg_pickle(ecg)  # 서버에서 ecg를 받아왔다고 가정, 정확히 어떤 형태로 받는지는 상관 없음, dictionary 든 array 든
            # ecg: np.ndarray with shape (12, 5000)
            out = ecg_infer_module.infer(ecg) # return dictionary
            print(out)  # 이 값을 다시 서버에 보내야 함
    return out

In [76]:
print(out)

NameError: name 'out' is not defined